-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

![](https://files.training.databricks.com/images/301/deployment_options_mllib.png)

There are **four main deployment options**:
* **Batch pre-compute**
* **Structured streaming**
* **Low-latency model serving**
* **Mobile/embedded (outside scope of class)**

We have already seen how to do batch predictions using Spark. Now let's look at how to make predictions on streaming data.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
 - **Apply a SparkML model on a simulated stream of data**

In [0]:
%run "../Includes/Classroom-Setup"

Defining courseware-specific utility methods...

The source for this dataset is
wasbs://courseware@dbacademy.blob.core.windows.net/scalable-machine-learning-with-apache-spark/v01/

Your dataset directory is
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/machine_learning/datasets

Skipping install of existing dataset.

## Load in Model & Data

We are loading in a repartitioned version of our dataset (100 partitions instead of 4) to see more incremental progress of the streaming predictions.

In [0]:
from pyspark.ml.pipeline import PipelineModel

pipeline_path = f"{datasets_dir}/airbnb/sf-listings/models/sf-listings-2019-03-06/pipeline_model"
pipeline_model = PipelineModel.load(pipeline_path)

repartitioned_path =  f"{datasets_dir}/airbnb/sf-listings/sf-listings-2019-03-06-clean-100p.parquet/"
schema = spark.read.parquet(repartitioned_path).schema

## Simulate streaming data

**NOTE**: You must specify a schema when creating a streaming source DataFrame.

In [0]:
streaming_data = (spark
                 .readStream
                 .schema(schema) # Can set the schema this way
                 .option("maxFilesPerTrigger", 1)
                 .parquet(repartitioned_path))

## Make Predictions

In [0]:
stream_pred = pipeline_model.transform(streaming_data)

**Let's save our results.**

In [0]:
import re

checkpoint_dir = working_dir + "/stream_checkpoint"
# Clear out the checkpointing directory
dbutils.fs.rm(checkpoint_dir, True) 

(stream_pred
 .writeStream
 .format("memory")
 .option("checkpointLocation", checkpoint_dir)
 .outputMode("append")
 .queryName("pred_stream")
 .start())

Out[10]: <pyspark.sql.streaming.StreamingQuery at 0x7f6a7fff6250>

In [0]:
untilStreamIsReady("pred_stream")

The stream is active and ready.

#### While this is running, take a look at the new Structured Streaming tab in the Spark UI.

In [0]:
display(
  sql("select * from pred_stream")
)

host_is_superhost,cancellation_policy,instant_bookable,host_total_listings_count,neighbourhood_cleansed,zipcode,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,minimum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,price,bedrooms_na,bathrooms_na,beds_na,review_scores_rating_na,review_scores_accuracy_na,review_scores_cleanliness_na,review_scores_checkin_na,review_scores_communication_na,review_scores_location_na,review_scores_value_na,host_is_superhostIndex,host_is_superhostOHE,cancellation_policyIndex,cancellation_policyOHE,instant_bookableIndex,instant_bookableOHE,neighbourhood_cleansedIndex,neighbourhood_cleansedOHE,zipcodeIndex,zipcodeOHE,property_typeIndex,property_typeOHE,room_typeIndex,room_typeOHE,bed_typeIndex,bed_typeOHE,features,prediction
t,moderate,t,1.0,Western Addition,94117,37.77149,-122.42872,Apartment,Private room,2.0,1.0,1.0,1.0,Real Bed,1.0,44.0,95.0,10.0,9.0,10.0,10.0,10.0,10.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Map(vectorType -> sparse, length -> 1, indices -> List(), values -> List())",1.0,"Map(vectorType -> sparse, length -> 5, indices -> List(1), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 1, indices -> List(), values -> List())",2.0,"Map(vectorType -> sparse, length -> 34, indices -> List(2), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 28, indices -> List(1), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 25, indices -> List(0), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 2, indices -> List(1), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 126, indices -> List(2, 9, 42, 69, 95, 96, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 37.77149, -122.42872, 2.0, 1.0, 1.0, 1.0, 1.0, 44.0, 95.0, 10.0, 9.0, 10.0, 10.0, 10.0, 10.0))",110.6106637326011
t,strict_14_with_grace_period,t,1.0,Outer Mission,94131,37.73285,-122.44235,House,Private room,3.0,1.0,1.0,0.0,Real Bed,2.0,240.0,97.0,10.0,9.0,10.0,10.0,10.0,9.0,119.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Map(vectorType -> sparse, length -> 1, indices -> List(), values -> List())",0.0,"Map(vectorType -> sparse, length -> 5, indices -> List(0), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 1, indices -> List(), values -> List())",16.0,"Map(vectorType -> sparse, length -> 34, indices -> List(16), values -> List(1.0))",9.0,"Map(vectorType -> sparse, length -> 28, indices -> List(9), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 25, indices -> List(1), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 2, indices -> List(1), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 126, indices -> List(1, 23, 50, 70, 95, 96, 100, 101, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 37.73285, -122.44235, 3.0, 1.0, 1.0, 2.0, 240.0, 97.0, 10.0, 9.0, 10.0, 10.0, 10.0, 9.0))",67.58495702355867
f,strict_14_with_grace_period,f,2.0,Mission,94110,37.75139,-122.41194,House,Entire home/apt,2.0,1.0,1.0,1.0,Real Bed,31.0,85.0,99.0,10.0,10.0,10.0,10.0,10.0,10.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 5, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 34, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 28, indices -> List(0), values -> List(1.0))",1.0,

In [0]:
display(
  sql("select count(*) from pred_stream")
)

count(1)
6940


Now that we are done, make sure to stop the stream

In [0]:
for stream in spark.streams.active:
    print(f"Stopping {stream.name}")
    stream.stop()                  # Stop the active stream
    try: stream.awaitTermination() # Wait for it to actually stop
    except: pass                   # Don't care if stopping fails

Stopping pred_stream

### What about Model Export?

* <a href="https://onnx.ai/" target="_blank">ONNX</a>
  * ONNX is very popular in the deep learning community allowing developers to switch between libraries and languages, but only has experimental support for MLlib.
* DIY (Reimplement it yourself)
  * Error-prone, fragile
* 3rd party libraries
  * See XGBoost notebook
  * <a href="https://www.h2o.ai/products/h2o-sparkling-water/" target="_blank">H2O</a>

### Low-Latency Serving Solutions

Low-latency serving can operate as quickly as tens to hundreds of milliseconds.  Custom solutions are normally backed by Docker and/or Flask (though Flask generally isn't recommended in production unless significant precations are taken).  Managed solutions also include:<br><br>

* <a href="https://docs.databricks.com/applications/mlflow/model-serving.html" target="_blank">MLflow Model Serving</a>
* <a href="https://azure.microsoft.com/en-us/services/machine-learning/" target="_blank">Azure Machine Learning</a>
* <a href="https://aws.amazon.com/sagemaker/" target="_blank">SageMaker</a>

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>